<a href="https://colab.research.google.com/github/ShenChikai/Aritificial-Intelligence/blob/main/NCF_Recommender_System/NCF_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 582 kB 30.1 MB/s 
     |████████████████████████████████| 136 kB 68.3 MB/s 
     |████████████████████████████████| 408 kB 68.3 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 1.1 MB 57.3 MB/s 
     |████████████████████████████████| 144 kB 66.9 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 71.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
np.random.seed(123)

# Dataset Cleaning

In [ ]:
raw_ratings = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Books_5.json', lines=True, nrows = 1000000)
raw_ratings.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A10000012B7CGYKOMPQ4L,000100039X,Adam,"[0, 0]",Spiritually and mentally inspiring! A book tha...,5,Wonderful!,1355616000,"12 16, 2012"
1,A2S166WSCFIFP5,000100039X,"adead_poet@hotmail.com ""adead_poet@hotmail.com""","[0, 2]",This is one my must have books. It is a master...,5,close to god,1071100800,"12 11, 2003"
2,A1BM81XB4QHOA3,000100039X,"Ahoro Blethends ""Seriously""","[0, 0]",This book provides a reflection that you can a...,5,Must Read for Life Afficianados,1390003200,"01 18, 2014"
3,A1MOSTXNIO5MPJ,000100039X,Alan Krug,"[0, 0]",I first read THE PROPHET in college back in th...,5,Timeless for every good and bad time in your l...,1317081600,"09 27, 2011"
4,A2XQ5LZHTD4AFT,000100039X,Alaturka,"[7, 9]",A timeless classic. It is a very demanding an...,5,A Modern Rumi,1033948800,"10 7, 2002"
5,A3V1MKC2BVWY48,000100039X,Alex Dawson,"[0, 0]",Reading this made my mind feel like a still po...,5,This book will bring you peace,1390780800,"01 27, 2014"
6,A12387207U8U24,000100039X,Alex,"[0, 0]","As you read, Gibran's poetry brings spiritual ...",5,Graet Work,1206662400,"03 28, 2008"
7,A29TRDMK51GKZR,000100039X,Alpine Plume,"[0, 0]","Deep, moving dramatic verses of the heart and ...",5,Such Beauty,1383436800,"11 3, 2013"
8,A3FI0744PG1WYG,000100039X,"Always Reading ""tkm""","[0, 0]",This is a timeless classic. Over the years I'...,5,The Prophet,1390953600,"01 29, 2014"
9,A2LBBQHYLEHM7P,000100039X,"Amazon Customer ""Full Frontal Nerdity""","[0, 0]",An amazing work. Realizing extensive use of Bi...,5,A Modern Classic,1379808000,"09 22, 2013"


In [ ]:
# Filter col
ratings = raw_ratings[["reviewerID", "asin", "overall", "reviewTime"]]
# Parse datetime
pd.options.mode.chained_assignment = None
ratings['reviewTime'] = pd.to_datetime(ratings['reviewTime'], format="%m %d, %Y")
# Rename col
ratings.rename(columns={'reviewerID': 'userid',
                   'asin': 'ASIN',
                   'overall': 'rating' ,
                   'reviewTime': 'timestamp',
                   },
          inplace=True, errors='raise')
ratings.head(10)

,userid,ASIN,rating,timestamp
0,A10000012B7CGYKOMPQ4L,000100039X,5,2012-12-16
1,A2S166WSCFIFP5,000100039X,5,2003-12-11
2,A1BM81XB4QHOA3,000100039X,5,2014-01-18
3,A1MOSTXNIO5MPJ,000100039X,5,2011-09-27
4,A2XQ5LZHTD4AFT,000100039X,5,2002-10-07
5,A3V1MKC2BVWY48,000100039X,5,2014-01-27
6,A12387207U8U24,000100039X,5,2008-03-28
7,A29TRDMK51GKZR,000100039X,5,2013-11-03
8,A3FI0744PG1WYG,000100039X,5,2014-01-29
9,A2LBBQHYLEHM7P,000100039X,5,2013-09-22


In [ ]:
#ratings = pd.read_csv('ratings_Digital_Music.csv', parse_dates=['timestamp'])

ratings['userid'] = pd.factorize(ratings['userid'])[0]
ratings['ASIN'] = pd.factorize(ratings['ASIN'])[0]

rand_userIds = np.random.choice(ratings['userid'].unique(), 
                                size=int(len(ratings['userid'].unique())*0.3), 
                                replace=False)

ratings = ratings.loc[ratings['userid'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

print(ratings.sample(5))

ratings['rank_latest'] = ratings.groupby(['userid'])['timestamp'].rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['userid', 'ASIN', 'rating']]
test_ratings = test_ratings[['userid', 'ASIN', 'rating']]

train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

There are 302664 rows of data from 95352 users
        userid   ASIN  rating  timestamp
533625  226097  14752       5 2013-02-07
645384  144547  18292       5 2010-08-25
504778  215873  12508       5 2011-11-05
625981  246010  17838       4 2012-08-08
835730    3133  23251       4 2010-07-16


,userid,ASIN,rating
655368,6371,18446,1
374539,7240,10118,1
44258,35308,698,1
983188,78734,28902,1
119404,28609,3224,1


In [ ]:
# Get a list of all movie IDs
all_movieIds = ratings['ASIN'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['userid'], train_ratings['ASIN']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userid'], ratings['ASIN']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=4)
        
num_users = ratings['userid'].max()+1
num_items = ratings['ASIN'].max()+1

print("Max value of userid is: ")
print(pd.unique(ratings['userid']).max())
print("Min value of userid is: ")
print(pd.unique(ratings['userid']).min())

print("Max value of ASIN is: ")
print(pd.unique(ratings['ASIN']).max())
print("Min value of ASIN is: ")
print(pd.unique(ratings['ASIN']).min())

all_movieIds = ratings['ASIN'].unique()

model = NCF(num_users, num_items, train_ratings, all_movieIds)

trainer = pl.Trainer(max_epochs=10,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)

# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['userid'], test_ratings['ASIN']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('userid')['ASIN'].apply(list).to_dict()

hits = []
for (u,i) in tqdm(test_user_item_set):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

  0%|          | 0/207312 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: Fals

Max value of userid is: 
317839
Min value of userid is: 
0
Max value of ASIN is: 
29238
Min value of ASIN is: 
0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

  0%|          | 0/95352 [00:00<?, ?it/s]

The Hit Ratio @ 10 is 0.50
